In [9]:
import numpy as np

# Constants representing the state of each cell
UNEXPLORED = -2
OBSTACLE = -1
SAFE = 0

# Initialize the global grid status as unexplored
global_grid = np.full((10, 10), UNEXPLORED)

def drone_scan(drone_pos, scan_range, actual_env):
    """
    Perform a local scan around a drone position.

    Args:
        drone_pos (tuple): Drone coordinates (x, y).
        scan_range (int): The range of the drone's field of view (e.g., 3 or 5).
        actual_env (np.array): The actual environment (10x10 matrix).

    Returns:
        tuple: local scanned information and top-left position of the scan.
    """
    half_range = scan_range // 2
    local_info = np.full((scan_range, scan_range), UNEXPLORED)

    for i in range(scan_range):
        for j in range(scan_range):
            global_x = drone_pos[0] - half_range + i
            global_y = drone_pos[1] - half_range + j

            # Check boundaries
            if 0 <= global_x < 10 and 0 <= global_y < 10:
                local_info[i, j] = actual_env[global_x, global_y]

    return local_info, (drone_pos[0] - half_range, drone_pos[1] - half_range)

def stitch_information(global_grid, local_info, top_left):
    """
    Merge local drone scans into the global coverage grid.

    Args:
        global_grid (np.array): Current global grid state.
        local_info (np.array): Local scan result from a drone.
        top_left (tuple): Top-left coordinate of the local scan in the global grid.

    Returns:
        np.array: Updated global grid.
    """
    # find top left corner of local scan for better aligning and stitching
    x_offset, y_offset = top_left

    for i in range(local_info.shape[0]):
        for j in range(local_info.shape[1]):
            x, y = x_offset + i, y_offset + j
            if 0 <= x < 10 and 0 <= y < 10:
                # Update global grid only if it's unexplored
                if global_grid[x, y] == UNEXPLORED:
                    global_grid[x, y] = local_info[i, j]
                # If conflicting info, prioritize safe information
                elif global_grid[x, y] != local_info[i, j]:
                    if local_info[i, j] == SAFE:
                        global_grid[x, y] = SAFE
    return global_grid

# Example actual environment (randomly generated obstacles and safe zones when searching)
actual_env = np.random.choice([OBSTACLE, SAFE], size=(10, 10), p=[0.2, 0.8])

In [10]:
# Example drones with position and scanning range
drones = [((1, 1), 3), ((4, 4), 5)]

# Perform local scans and stitch them into global grid
for pos, scan_range in drones:
    local_info, top_left = drone_scan(pos, scan_range, actual_env)
    global_grid = stitch_information(global_grid, local_info, top_left)

print("Final Global Grid after Stitching:")
print(global_grid)

Final Global Grid after Stitching:
[[-1  0  0 -2 -2 -2 -2 -2 -2 -2]
 [ 0 -1  0 -2 -2 -2 -2 -2 -2 -2]
 [ 0 -1  0  0  0  0  0 -2 -2 -2]
 [-2 -2 -1  0  0  0 -1 -2 -2 -2]
 [-2 -2 -1 -1  0  0  0 -2 -2 -2]
 [-2 -2 -1  0  0  0  0 -2 -2 -2]
 [-2 -2  0  0  0  0  0 -2 -2 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2]
 [-2 -2 -2 -2 -2 -2 -2 -2 -2 -2]]
